In [10]:
import requests
import pandas as pd

from fake_useragent import UserAgent

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

In [1]:
# Определяем списки тегов и типов объектов
amenities = [  # Список типов объектов (магазины, кафе и т.д.) для поиска с помощью Nominatim
    "restaurant",
    "cafe",
    "fast_food",
    "fuel",  # Автозаправки
    "hospital",
    "pharmacy",
    "school",
    "kindergarten",
    "post_office",
    "bank",
    "atm",
    "supermarket",
    "convenience",  # Магазин шаговой доступности
    "police",
    "fire_station",
    "community_centre",  # Дом культуры, общественный центр
    "place_of_worship",
    "library",
    "townhall",  # Администрация
    "doctors",  # Врачи
    "dentist",  # Стоматолог
    "clinic",  # Поликлиника
    "courthouse",  # Суд
    "theatre",  # Театр
    "cinema",  # Кинотеатр
    "nightclub",  # Ночной клуб
    "pub",  # Паб
    "bar",  # Бар
    "marketplace",  # Рынок
    "recycling",  # Переработка отходов
    "waste_disposal",  # Утилизация отходов
    "toilets",  # Туалеты
    "drinking_water",  # Питьевая вода
    "charging_station",  # Зарядная станция (электромобили)
    "parking",  # Парковка
    "bus_station",  # Автобусная станция
    "taxi",  # Такси
    "car_wash",  # Автомойка
    "car_repair",  # Автосервис
    "veterinary",  # Ветеринар
    "social_facility",  # Объекты социальной помощи
    "childcare",  # Детский сад (другое обозначение)
    "government",
    "pharmacy",
]

In [2]:
offices = ["townhall", "government"]  # Список офисов (для поиска через Overpass)
shops = ["*"]  # Список магазинов (для поиска через Overpass)
crafts = ["*"]  # Список ремесленных мастерских (для поиска через Overpass)
sports = ["*"]  # Список спортивных объектов (для поиска через Overpass)
healthcares = ["*"]  # Список объектов здравоохранения (для поиска через Overpass)

In [3]:
tags = {  # Словарь тегов для поиска через Overpass API
    "office": offices,
    "shop": shops,
    "craft": crafts,
    "sport": sports,
    "healthcare": healthcares,
}

In [4]:
# Создаем пустой список для хранения данных
data = []

In [5]:
# Определяем Bounding Box для Яр-Сале
# Координаты взяты отсюда: https://www.openstreetmap.org/export
yar_sale_bbox = [70.6503, 66.8348, 71.0094, 66.9178]

In [ ]:
# Функция для запроса данных из Overpass API
def query_overpass(bbox, key, value):
    """
    Запрашивает данные из Overpass API.

    Args:
        bbox: Bounding box (min_lon, min_lat, max_lon, max_lat).
        key: Ключ тега (например, "amenity").
        value: Значение тега (например, "restaurant").

    Returns:
        JSON-ответ от Overpass API или None в случае ошибки.
    """
    overpass_url = "http://overpass-api.de/api/interpreter" 
    query = f""" 
    [out:json]; 
    (node 
      ["{key}"="{value}"] 
      ({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]}); 
     way 
      ["{key}"="{value}"] 
      ({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]}); 
     relation 
      ["{key}"="{value}"] 
      ({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]}); 
    ); 
    out center; 
    """ 
    try: 
        response = requests.get(overpass_url, params={'data': query}) 
        response.raise_for_status()  # Проверка на HTTP ошибки 
        return response.json() 
    except requests.exceptions.RequestException as e: 
        print(f"Ошибка при запросе Overpass API для {key}={value}: {e}") 
        return None


In [7]:
# Функция для обработки элемента ответа Overpass API
def parse_response_element(element):
    """
    Разбирает элемент ответа Overpass API и извлекает информацию об адресе и координатах.

    Args:
        element: Элемент из ответа Overpass API.

    Returns:
        tuple: Кортеж (full_address, lat, lon).  Если данные не найдены, возвращает (None, None, None).
    """
    full_address = None
    lat = None
    lon = None
    tags = element.get("tags")
    if tags:
        city = tags.get("addr:city", "")
        housenumber = tags.get("addr:housenumber", "")
        street = tags.get("addr:street", "")
        name = tags.get("name", "")
        # Создаем список частей адреса, отфильтровывая пустые значения
        address_parts = [part for part in [name, housenumber, street, city] if part]
        full_address = ", ".join(address_parts)

    center = element.get("center")
    if center:  # Проверяем, что 'center' существует
        lat = center.get("lat")
        lon = center.get("lon")

    return full_address, lat, lon

In [8]:
# Функция для получения данных из Overpass API по тегам
def get_data_by_tag(tag_key, tag_value, bbox):
    """
    Получает данные из Overpass API по заданным тегам.

    Args:
        tag_key: Ключ тега (например, "amenity").
        tag_value: Значение тега (например, "restaurant").
        bbox: Bounding box.

    Returns:
        Список словарей с данными об объектах.
    """
    data = []  # Создаем список для хранения данных
    overpass_data = query_overpass(bbox, tag_key, tag_value)
    if overpass_data and "elements" in overpass_data:
        for element in overpass_data["elements"]:
            try:
                full_address, lat, lon = parse_response_element(element)
                if lat is not None and lon is not None:  # Проверяем, что координаты получены
                    data.append({
                        "address": full_address,
                        "X": lon,  # Долгота (longitude)
                        "Y": lat,  # Широта (latitude)
                        "type": f"{tag_key} {tag_value}",  # Комбинируем ключ и значение тега
                        "source": "Overpass API",
                    })
            except Exception as e:
                print(f"Ошибка обработки элемента Overpass API: {e}")
    return data


In [11]:
# Инициализация Geolocator (Nominatim)
geolocator = Nominatim(user_agent=UserAgent().random)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # Ограничение скорости запросов

In [12]:
# Определяем границы Яр-Сале (для Nominatim)
yar_sale_area = 'Яр-Сале, Yamalsky Rayon, Yamalo-Nenets Autonomous Okrug, Ural Federal District, 629700, Russia'

In [13]:
# Цикл по типам объектов (amenities) для Nominatim
for object_type in amenities:
    """
    Поиск объектов по типам (amenity) с использованием Nominatim.
    """
    query = f"{object_type} in {yar_sale_area}"  # Формируем запрос
    try:
        locations = geocode(query, timeout=10, exactly_one=False)
        if locations:
            print(f"Найдено несколько объектов типа '{object_type}' в Яр-Сале через Nominatim.")
            for location in locations:  # Перебираем все найденные location
                if location:  # Проверяем на null
                    address = location.address
                    lat, lon = location.latitude, location.longitude
                    data.append({
                        "address": address,
                        "X": lon,  # Долгота (longitude)
                        "Y": lat,  # Широта (latitude)
                        "type": f"amenity {object_type}",  # Формируем тип объекта
                        "source": "Nominatim",
                    })
        else:
            print(f"Объекты типа '{object_type}' не найдены в Яр-Сале через Nominatim.")
    except GeocoderTimedOut:
        print(f"Timeout при запросе {query}")
        time.sleep(5)  # Пауза перед повторной попыткой
    except GeocoderServiceError as e:
        print(f"Ошибка сервиса при запросе {query}: {e}")
    except Exception as e:
        print(f"Неизвестная ошибка при запросе {query}: {e}")


Найдено несколько объектов типа 'restaurant' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'cafe' в Яр-Сале через Nominatim.
Объекты типа 'fast_food' не найдены в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'fuel' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'hospital' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'pharmacy' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'school' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'kindergarten' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'post_office' в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'bank' в Яр-Сале через Nominatim.
Объекты типа 'atm' не найдены в Яр-Сале через Nominatim.
Объекты типа 'supermarket' не найдены в Яр-Сале через Nominatim.
Объекты типа 'convenience' не найдены в Яр-Сале через Nominatim.
Найдено несколько объектов типа 'police' в Яр-Сале через Nominatim.
Объекты типа 'fire_station' не найдены в Яр-Сале через No

In [14]:
# Цикл по тегам (ключ-значение) для Overpass API
for key in tags:
    """
    Поиск объектов по тегам с использованием Overpass API.
    """
    for value in tags[key]:
        data.extend(get_data_by_tag(key, value, yar_sale_bbox))


In [22]:
# Создаем DataFrame
df = pd.DataFrame(data)


In [23]:
df

,address,X,Y,type,source
0,"Ялэмд, 4, Советская улица, Промзона Южная, Яр-...",70.844429,66.859093,amenity restaurant,Nominatim
1,"Кафе ""АЙС"", 33, улица Мира, Яр-Сале, муниципал...",70.839189,66.866210,amenity cafe,Nominatim
2,"Жар-Пицца, 19, Советская улица, Промзона Южная...",70.841197,66.861311,amenity cafe,Nominatim
3,"Советская улица, Промзона Южная, Яр-Сале, муни...",70.830478,66.869964,amenity fuel,Nominatim
4,"Туберкулёзное отделение, улица Худи Сэроко, Яр...",70.831499,66.867709,amenity hospital,Nominatim
5,"Роддом, 21А, улица Худи Сэроко, Яр-Сале, муниц...",70.839417,66.860087,amenity hospital,Nominatim
6,"Яр-Салинская центральная районная больница, ул...",70.839962,66.859341,amenity hospital,Nominatim
7,"Стационар, 19, улица Худи Сэроко, Яр-Сале, мун...",70.840313,66.859180,amenity hospital,Nominatim
8,"31, Советская улица, Промзона Южная, Яр-Сале, ...",70.838928,66.863341,amenity pharmacy,Nominatim
9,"Градусник, улица Худи Сэроко, Яр-Сале, муницип...",70.836634,66.861959,amenity pharmacy,Nominatim


In [26]:
if not df.empty:
    """
    Удаление дубликатов в DataFrame.
    """
    # Удаление дубликатов, учитывая нормализованный адрес, координаты и тип.
    df_cleaned = df.sort_values(by=['address'], key=lambda x: x.str.len(), ascending=False, 
        ignore_index=True).drop_duplicates(subset=['X', 'Y', 'type'], keep='first')
    path = './data/yar_sale_objects.csv'
    df_cleaned.to_csv(path, index=False)
    print(f"Данные сохранены в {path}")
else:
    print("Нет данных для сохранения.")

Данные сохранены в ./data/yar_sale_objects.csv


In [27]:
df_cleaned.head()

,address,X,Y,type,source
0,"Храм Блаженной Ксении Петербургской, 2, Советс...",70.844832,66.858587,amenity place_of_worship,Nominatim
1,"Детский сад ""Солнышко"" (корпус-1), Советская у...",70.843028,66.859235,amenity kindergarten,Nominatim
2,"Детский сад ""Солнышко"" (корпус-2), Советская у...",70.835861,66.868523,amenity kindergarten,Nominatim
3,"Центр национальных культур, 17, Советская улиц...",70.841149,66.860969,amenity community_centre,Nominatim
4,"Яр-Салинская центральная районная больница, ул...",70.839962,66.859341,amenity hospital,Nominatim
